In [9]:
import pandas as pd
import numpy as np
import json
import os
import uuid


from sentence_transformers import SentenceTransformer, util

import elasticsearch
from elasticsearch import Elasticsearch
from elasticsearch import helpers

from tqdm.auto import tqdm
tqdm.pandas()

In [10]:
%%time
df  = pd.read_csv("data job posts.csv")

Wall time: 2.39 s


In [12]:
class Tokenizer(object):
    def __init__(self):
        self.model = SentenceTransformer('all-MiniLM-L6-v2')

    def get_token(self, documents):
        sentences  = [documents]
        sentence_embeddings = self.model.encode(sentences)
        encod_np_array = np.array(sentence_embeddings)
        encod_list = encod_np_array.tolist()
        return encod_list[0]


In [13]:
token_instance = Tokenizer()

In [14]:
df = df.head(5000)
df = df.dropna(how='all')

In [18]:
df['vector'] = df['jobpost'].progress_apply(token_instance.get_token)

  0%|          | 0/5000 [00:00<?, ?it/s]

```sh
PUT posting1 
{
   "mappings":{
      "properties":{
         "vector":{
            "type":"dense_vector",
            "dims":384,
            "index": true,
            "similarity": "cosine"
         }
      }
   }
}

```

In [19]:
elk_data = df.to_dict("records")

In [31]:
ENDPOINT = 'https://localhost:9200/'
USERNAME = 'elastic'
PASSWORD = 'vOfdH9D7uTx4hSZ7mAc0'

In [32]:
es = Elasticsearch(
    hosts=[ENDPOINT],
    ca_certs = 'D:\work\es\elasticsearch-8.4.0\config\certs\http_ca.crt',
    basic_auth=(USERNAME, PASSWORD)
)
es.ping()

True

In [39]:
doc_id = 0
for x in elk_data:
    doc_id += 1
    try:
        doc = {
           "title":x.get("Title", ""),
            "company":x.get("Company", ""),
            "location":x.get("Location", ""),
            "salary":x.get("Salary", ""),
            "vector":x.get("vector", ""),
            "job_description":x.get("JobDescription", ""),

        }
        es.index(index='posting1', id=doc_id, document=doc)
    except Exception as e: pass

In [85]:
#INPUT = input("Enter the Input Query ")
INPUT = "someone that knows Java or C++"
token_vector = token_instance.get_token(INPUT)

knn = {
    "field": "vector",
    "query_vector": token_vector,
    "k": 10,
    "num_candidates": 10
}

q = {
    "match": {
      "title": INPUT
    }
}

res = es.search(index='posting1', knn=knn, _source=["title","company"])
docs = [x['_source'] for x in res['hits']['hits']] 
docs

[{'company': 'CQG Yerevan', 'title': 'C++ Senior Software Developer (UNIX)'},
 {'company': 'CQG Yerevan', 'title': 'C++ Senior Software Developer (UNIX)'},
 {'company': 'CQG Yerevan', 'title': 'C++ Senior Software Developer (UNIX)'},
 {'company': 'CQG-Yerevan', 'title': 'C++ Senior Software Developer (UNIX)'},
 {'company': 'Lycos Europe',
  'title': 'Java or C++ backend developer for Lycos Communities'},
 {'company': 'CQG-Yerevan', 'title': 'C++ Senior Software Developer (UNIX)'},
 {'company': 'ISMO Tech CJSC', 'title': 'C++ Software Developer'},
 {'company': 'CQG-Yerevan', 'title': 'C++ Senior Software Developer (UNIX)'},
 {'company': 'Vested Development, Inc.',
  'title': 'Java Senior Developer / Architect'},
 {'company': 'Vested Development, Inc.',
  'title': 'Java Senior Developer/ Architect'}]

In [87]:
res = es.search(index='posting1',query=q,knn=knn, _source=["title","company"])
docs = [x['_source'] for x in res['hits']['hits']] 
docs

[{'company': 'Lycos Europe',
  'title': 'Java or C++ backend developer for Lycos Communities'},
 {'company': 'Lycos Europe', 'title': 'Java/C++ Backend Developers'},
 {'company': 'Lycos Europe', 'title': 'Java/C++/PHP Software Developers'},
 {'company': 'LVFH', 'title': 'Senior or Intermediate Software Engineer'},
 {'company': 'Lycos Armenia CJSC',
  'title': 'Java/C++ Developer for Lycos Mail'},
 {'company': 'Lycos Europe',
  'title': 'Java/C++ Developer for Lycos Communities'},
 {'company': 'CQGI MA', 'title': 'C++/ C# Senior Developer'},
 {'company': 'CQGI MA', 'title': 'C++/ C# Senior Developer'},
 {'company': 'Lycos Europe', 'title': 'C/C++ Software  Developers'},
 {'company': 'Lycos Europe', 'title': 'C/C++ Software  Developers'}]